<a href="https://colab.research.google.com/github/rahul88b/startt/blob/master/Mafat_color.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import matplotlib
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import scipy.misc
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import zipfile
from sklearn import preprocessing
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Add, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Concatenate
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input


from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/gdrive/My\ Drive/foo.txt

Hello Google Drive!

In [0]:
train = pd.read_csv("./gdrive/My Drive/mafat/train.csv")
train.set_index('image_id',inplace=True)

In [0]:
image_size=64
num_channels=3
num_pictures=275
train_dir="./gdrive/My Drive/mafat/training imagery/"
val_dir="./gdrive/My Drive/mafat/test imagery/"


In [6]:
image_name=[]
num_tiff=0
for file in os.listdir(train_dir):
  if file[-3:]!='jpg':
    num_tiff=num_tiff+1
    continue
  image_name.append(file[:-4])
print(num_tiff)

80


In [7]:
total_pictures=len(image_name)
total_pictures

1583

In [0]:
#importing data

def get_next_batch(image_index):
  
  images=np.ndarray(shape=(num_pictures,image_size,image_size,num_channels))
  current_index=0
  labels=[]
  num_tiff=0

  while current_index<num_pictures and image_index<total_pictures:
    
      file=image_name[image_index]
    
      image=os.path.join(train_dir,file+'.jpg')
      file_data=mpimg.imread(image)
      height=file_data.shape[0]
      breadth=file_data.shape[1]
    
      image_id=np.ndarray(shape=(1),dtype='int32')
      image_id=int(file)
    
    
      lb=train.loc[image_id,['color']]
      label_gate=0
      if(lb.shape[0]==1):
          lb=lb.values
          label_gate=1
      else:
          lb=lb['color'].tolist()
    
    
      bb=train.loc[image_id,['p1_x','p_1y',' p2_x',' p2_y',' p3_x',' p3_y',' p4_x',' p4_y']].values
      n=bb.shape[0]
      if len(bb.shape)==1:
        box=np.ndarray(shape=(1,8))
        box[0,:]=bb[0]
        bb=box
        n=1
    
      box=np.ndarray(shape=(n,4))
      for i in range(n):
        x1=min(bb[i,0],bb[i,2],bb[i,4],bb[i,6])
        x2=max(bb[i,0],bb[i,2],bb[i,4],bb[i,6])
        y1=min(bb[i,1],bb[i,3],bb[i,5],bb[i,7])
        y2=max(bb[i,1],bb[i,3],bb[i,5],bb[i,7])
        box[i,0]=y1/height
        box[i,1]=x1/breadth
        box[i,2]=y2/height
        box[i,3]=x2/breadth
        y=np.ndarray(shape=(1,file_data.shape[0],file_data.shape[1],3))
        y[0,:,:,:]=file_data[:,:,0:3]
   
      cropped=tf.image.crop_and_resize(image=y,boxes=box,box_ind=[0]*box.shape[0],crop_size=[image_size,image_size],method='bilinear')
      sess=tf.Session()
      with sess.as_default():
          tensor=cropped
          cropped=tensor.eval()
      n=cropped.shape[0]
    
    
      for i in range(n):
          images[current_index,:,:,:]=cropped[i,:,:,:]
          
          current_index=current_index+1
          if label_gate==0:
            labels.append(lb[i])
          else:
            labels.extend(lb)

          if current_index>=num_pictures:
            break
      image_index=image_index+1
  
  labels=np.asarray(labels)
  #shuffling data

  random=np.arange(num_pictures)
  np.random.shuffle(random)
  images=images[random]
  labels=labels[random]
  
  
  return images[:250,:,:,:],labels[:250],image_index,images[250:,:,:,:],labels[250:]




In [0]:
all_labels=train['color'].values
all_labels=np.asarray(all_labels)
le = preprocessing.LabelEncoder()
training_le = le.fit(all_labels)
num_classes=le.classes_.shape[0]

In [0]:
def get_encoded_label(labels,val_y):
  return training_le.transform(labels),training_le.transform(val_y)

In [11]:
le.classes_

array(['black', 'blue', 'green', 'other', 'red', 'silver/grey', 'white',
       'yellow'], dtype=object)

In [0]:
def identity_block(X, f, filters):
    
    F1, F2, F3 = filters
     
    X_shortcut = X
   
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding='same',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = X = Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)

    X = layers.Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [0]:

def convolutional_block(X, f, filters,s = 2):
    
    F1, F2, F3 = filters
    
    X_shortcut = X

    X = Conv2D(F1, (1, 1), strides = (s,s), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F2,(f,f),strides=(1,1),padding='same',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(F3,(1,1),strides=(1,1),padding='valid',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)

    X_shortcut = Conv2D(F3,(1,1),strides=(s,s),padding='valid',kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    X = layers.Add()([X,X_shortcut])
    X = Activation('relu')(X)
    return X

In [0]:

def ResNet50(input_shape, classes):
    
    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)
    
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    X = convolutional_block(X,f=3,filters=[128,128,512],s=2)
    X = identity_block(X,3,[128,128,512])
    X = identity_block(X,3,[128,128,512])
    X = identity_block(X,3,[128,128,512])

    X = convolutional_block(X,f=3,filters=[256,256,1024],s=2)
    X = identity_block(X,3,[256,256,1024])
    X = identity_block(X,3,[256,256,1024])
    X = identity_block(X,3,[256,256,1024])
    X = identity_block(X,3,[256,256,1024])
    X = identity_block(X,3,[256,256,1024])

    X = convolutional_block(X,f=3,filters=[512,512,2048],s=2)
    X = identity_block(X,3,[512,512,2048])
    X = identity_block(X,3,[512,512,2048])

    X = AveragePooling2D((2,2))(X)
    
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    model = Model(inputs = X_input, outputs = X)

    return model

In [0]:
model = ResNet50(input_shape = (image_size, image_size, 3), classes=num_classes)

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
num_times=8
image_index=0
X_val=np.ndarray(shape=(0,image_size,image_size,num_channels))
Y_val=np.ndarray(shape=(0))

for i in range(num_times):
  
  X,labels,image_index,val_x,val_y=get_next_batch(image_index)
  print('image_index',image_index)
  Y,val_y=get_encoded_label(labels,val_y)
  X_val=np.concatenate((X_val,val_x), axis=0)
  Y_val=np.concatenate((Y_val,val_y), axis=0)
  model.fit(X, Y, epochs = 4, batch_size = 32)

image_index 63
Epoch 1/4
250/250 [==============================] - 13s 51ms/step - loss: 3.2650 - acc: 0.2440
Epoch 2/4
250/250 [==============================] - 2s 8ms/step - loss: 1.4876 - acc: 0.5320
Epoch 3/4
250/250 [==============================] - 2s 8ms/step - loss: 1.2319 - acc: 0.5480
Epoch 4/4
250/250 [==============================] - 2s 8ms/step - loss: 1.1365 - acc: 0.6080
image_index 131
Epoch 1/4
250/250 [==============================] - 3s 12ms/step - loss: 1.6281 - acc: 0.4920
Epoch 2/4
250/250 [==============================] - 2s 9ms/step - loss: 1.3895 - acc: 0.5960
Epoch 3/4
250/250 [==============================] - 2s 8ms/step - loss: 1.2764 - acc: 0.6800
Epoch 4/4
250/250 [==============================] - 2s 8ms/step - loss: 1.0937 - acc: 0.7320
image_index 183
Epoch 1/4
250/250 [==============================] - 2s 9ms/step - loss: 1.7729 - acc: 0.5280
Epoch 2/4
250/250 [==============================] - 2s 8ms/step - loss: 1.5691 - acc: 0.5840
Epoch 3/4


In [22]:
val_set = model.evaluate(X_val, Y_val)
print ("Loss = " + str(val_set[0]))
print ("Test Accuracy = " + str(val_set[1]))

200/200 [==============================] - 2s 9ms/step
Loss = 1.787266550064087
Test Accuracy = 0.615


In [0]:
model.save("./gdrive/My Drive/mafat/color/model_color")

In [0]:
model=load_model("./gdrive/My Drive/mafat/color/model_color")

In [0]:
model.save("./gdrive/My Drive/mafat/color/MyModel_color")